## 1. Find Header_Tags Of Wikipedia

In [139]:
def find_header_tag(url):
    from bs4 import BeautifulSoup
    import requests
    import pandas as pd
    
#Now, we will send get request to the webpage server to get the source code of the page
#which is to be scraped by using the request library
#page = requests.get("https://en.wikipedia.org/wiki/Main_Page")
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    header_tags = []
    for i in soup.find_all('span', class_='mw-headline'):
        header_tags.append(i.get_text().replace('\n', ''))
    htags = pd.DataFrame({})
    htags['Header Tags'] = header_tags
    return htags

In [140]:
find_header_tag('https://en.wikipedia.org/wiki/Main_Page')

,Header Tags
0,From today's featured article
1,Did you know ...
2,In the news
3,On this day
4,Today's featured picture
5,Other areas of Wikipedia
6,Wikipedia's sister projects
7,Wikipedia languages


### Common Function to Scrap IMDB site

In [142]:
def top100Movies_IMDB(url):
    from bs4 import BeautifulSoup
    import requests
    import pandas as pd
    
#Getting Data Fromm Webpage, for this we studied website and came to know that..
#advanced search will help to get the below url
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    Movie_Names, Year_Of_Release, Ratings = [],[],[]
    
    for i in soup.find_all('h3', class_ = 'lister-item-header'):
            Movie_Names.append(i.a.text)
    for i in soup.find_all('span', class_ = 'lister-item-year text-muted unbold'):
        Year_Of_Release.append(''.join(filter(lambda j: j.isdigit(), i.text.replace('\n','').replace(' ', ''))))
    Year_Of_Release = [sub.replace('(', '').replace(")","").replace('-','') for sub in Year_Of_Release]
    for i in soup.find_all('div', class_ = 'inline-block ratings-imdb-rating'):
        Ratings.append(i.strong.text)

#Values into Data Frame:-
    Movie_lst = pd.DataFrame({})
    Movie_lst['Movie Name'] = pd.Series(Movie_Names)
    Movie_lst['Movie Release Year'] = pd.Series(Year_Of_Release)
    Movie_lst['Movie Rating'] = pd.Series(Ratings)
    Movie_lst = Movie_lst.sort_values(by=['Movie Rating'], ascending = False)
    Movie_lst = Movie_lst.head(100)

#Data Frame to CSV export(zip file):-
    if ("countries=in" in url):
        compression_opts = dict(method='zip', archive_name='TopMovies_Indian.csv')
        return Movie_lst.to_csv('TopMovies_Indian.zip', index=False, compression=compression_opts)
    else:
        compression_opts = dict(method='zip', archive_name='TopMovies.csv')
        return Movie_lst.to_csv('TopMovies.zip', index=False, compression=compression_opts)

## 2. Find Top 100 Movies From IMDB site

In [143]:
def find_top100Movies(url):
    url = url+'/search/title/?title_type=feature&sort=num_votes,desc&count=250'
    return top100Movies_IMDB(url)

In [144]:
find_top100Movies('https://www.imdb.com')

## 3. Find Top 100 Indian Movies in IMDB site

In [145]:
def find_top100Movies_Indian(url):
    url = url+'/search/title/?title_type=feature&countries=in&adult=include&sort=num_votes,desc&count=250'
    return top100Movies_IMDB(url)

In [146]:
find_top100Movies_Indian('https://www.imdb.com')

## 4. Book name, author name, genre and book review of any 5 books from ‘www.bookpage.com’

In [151]:
def find_reviews(url):
    from bs4 import BeautifulSoup
    import requests
    import pandas as pd
    Book_Names, Author_Names, Genre, Book_Reviews, review_links = [],[],[],[],[]
#On study, for this site there is seperate webpage to get reviews to simplify that....
#we create another function to get review data onto single list
    review_links = get_review_links(url)
    
    for i in review_links:
        page_review = requests.get(url+i)
        soup = BeautifulSoup(page_review.content, 'html.parser')
        Name,Author,Genres,review = [],[],[],[]
        
        for j in soup.find_all('h1', class_ = 'italic'):
            Book_Names.append(j.get_text().replace('\n',''))
        for j in soup.find_all('h4', class_ = 'sans'):
            Author_Names.append(j.text.replace('\n',''))
        for j in soup.find_all('p', class_ = 'genre-links'):
            Genre.append(j.a.text)
        review_temp = []
        for j in soup.find_all('div', class_ = 'article-body'):
            children = j.findChildren('p', recursive=False)
            for child in children:
                review_temp.append(child.text.replace('\n',''))
            Book_Reviews.append(review_temp)
            
#Values into Data Frame:-
    Book_lst = pd.DataFrame({})
    Book_lst['Book Name'] = pd.Series(Book_Names) #To handle NaN values, we converted into Series
    Book_lst['Author Name'] = pd.Series(Author_Names)
    Book_lst['Genre'] = pd.Series(Genre)
    Book_lst['Book Review'] = pd.Series(Book_Reviews)
    Book_lst = Book_lst.head()
    Book_lst
#Data Frame to CSV export(zip file):-
    compression_opts = dict(method='zip', archive_name='Book_Reviews.csv')
    return Book_lst.to_csv('Book_Reviews.zip', index=False, compression=compression_opts)  


#function to get review links of different pages
def get_review_links(url):
    url = url+'/reviews'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    td = soup.find_all('div', class_='read-full')
    links =[]
    for i in td:
        children = i.findChildren("a" , recursive=True)
        for child in children:
            links.append(child.get('href'))
    return links

In [152]:
find_reviews('https://bookpage.com')

### Below are Common Function to scrap ICC Cricket Site as per requirement

#This Function is to get top teams from both men and women cricket team from ‘www.icc-cricket.com’
#depends upon url passing through its parent funcion

In [154]:
#This Function is to get top teams from both men and women cricket team from ‘www.icc-cricket.com’
#depends upon url passing through its parent funcion
def Top10_teams(url):
    from bs4 import BeautifulSoup
    import requests
    import pandas as pd  
    Team_Position, Team_Name, No_of_Matches, Points, Rating = [],[],[],[],[]    
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    position = soup.find_all('td', {'class': ['rankings-block__banner--pos','table-body__cell table-body__cell--position u-text-right']})
    t_name = soup.find_all('span', class_='u-hide-phablet')
    match = soup.find_all('td',{'class':['table-body__cell u-center-text', 'rankings-block__banner--matches','rankings-block__banner--points']})
    rate = soup.find_all('td',{'class':['rankings-block__banner--rating u-text-right', 'table-body__cell u-text-right rating']})
    
    for i in position:
        Team_Position.append(i.text.replace('\n',''))
    for i in t_name:
        Team_Name.append(i.text.replace('\n',''))
    for i, j in enumerate(match):
        if i % 2 ==0:
            No_of_Matches.append(j.text.replace('\n',''))
        else:
            Points.append(j.text.replace('\n',''))
    for i in rate:
        Rating.append(i.text.replace('\n','').replace(' ', ''))

    #Values into Data Frame:-    
    Team_lst = pd.DataFrame({})
    Team_lst['Team Position'] = pd.Series(Team_Position) #To handle NaN values(If there is any), we converted into Series
    Team_lst['Team Name'] = pd.Series(Team_Name)
    Team_lst['No.Of Matches'] = pd.Series(No_of_Matches)
    Team_lst['Points'] = pd.Series(Points)
    Team_lst['Rating'] = pd.Series(Rating)
    Team_lst = Team_lst.head(10)
    Team_lst

    #Data Frame to CSV export(zip file):-
    if ("womens" in url):
        compression_opts = dict(method='zip', archive_name='Top10_ODI_Womens_Team.csv')
        return Team_lst.to_csv('Top10_ODI_Womens_Team.zip', index=False, compression=compression_opts)
    elif ("mens" in url):
        compression_opts = dict(method='zip', archive_name='Top10_ODI_Mens_Team.csv')
        return Team_lst.to_csv('Top10_ODI_Mens_Team.zip', index=False, compression=compression_opts)

#This Function is to get top players from both batting, bowling, men and womens team from ‘www.icc-cricket.com’
#depends upon url passing through its parent funcion

In [190]:
def find_Top10_player(url):
    from bs4 import BeautifulSoup
    import requests
    import pandas as pd
    Player_Position, Player_Name, Team, Rating, Career_best_rating = [],[],[],[],[]
    
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    position = soup.find_all('td', {'class': ['rankings-block__position','table-body__cell table-body__cell--position u-text-right']})
    p_name = soup.find_all('div', class_='rankings-block__banner--name-large')
    p_name1 = soup.find_all('td', class_='table-body__cell rankings-table__name name')
    match = soup.find_all('span',class_='table-body__logo-text')
    match1 = soup.find_all('div', class_= 'rankings-block__banner--nationality')
    rate = soup.find_all('div',class_='rankings-block__banner--rating')
    rate1 = soup.find_all('td',class_='table-body__cell rating')
    best = soup.find_all('span',class_='rankings-block__career-best-text')
    best1 = soup.find_all('td',class_='table-body__cell u-text-right u-hide-phablet')
 
    for i in position:
        Player_Position.append(''.join(filter(lambda j: j.isdigit(), i.text.replace('\n','').replace(' ', ''))))
    for i in p_name:
        Player_Name.append(i.text.replace('\n',''))
    for i in p_name1:
        Player_Name.append(i.a.text.replace('\n',''))
    for i in (match1+match):
        Team.append(i.text.replace('\n',''))
    for i in (rate+rate1):
        Rating.append(i.text.replace('\n',''))
    for i in (best+best1):
        Career_best_rating.append(i.text.replace('\n','').replace(' ', ''))
        
#Values into Data Frame:-    
    player_lst = pd.DataFrame({})
    player_lst['Position'] = pd.Series(Player_Position) #To handle NaN values(If there is any), we converted into Series
    player_lst['Player Name'] = pd.Series(Player_Name)
    player_lst['Player Team'] = pd.Series(Team)
    player_lst['Rating'] = pd.Series(Rating)
    player_lst['Career best rating'] = pd.Series(Career_best_rating)
    player_lst = player_lst.head(10)
    player_lst
    
#Data Frame to CSV export(zip file):-
    if (("batting" in url) and("/mens/" in url)):
        compression_opts = dict(method='zip', archive_name='Top10_ODI_Mens_Batting.csv')
        return player_lst.to_csv('Top10_ODI_Mens_Batting.zip', index=False, compression=compression_opts)
    elif (("batting" in url) and("/womens/" in url)):
        compression_opts = dict(method='zip', archive_name='Top10_ODI_womens_Batting.csv')
        return player_lst.to_csv('Top10_ODI_womens_Batting.zip', index=False, compression=compression_opts) 
    elif (("bowling" in url) and("/mens/" in url)):
        compression_opts = dict(method='zip', archive_name='Top10_ODI_Mens_Bowling.csv')
        return player_lst.to_csv('Top10_ODI_Mens_Bowling.zip', index=False, compression=compression_opts)
    elif (("bowling" in url)and("/womens/" in url)):
        compression_opts = dict(method='zip', archive_name='Top10_ODI_womens_Bowling.csv')
        return player_lst.to_csv('Top10_ODI_womens_Bowling.zip', index=False, compression=compression_opts)
    elif (("/womens/" in url)and("all-rounder" in url)):
        compression_opts = dict(method='zip', archive_name='Top10_ODI_womens_allrounder.csv')
        return player_lst.to_csv('Top10_ODI_womens_allrounder.zip', index=False, compression=compression_opts)

## 5. Python program to scrape cricket rankings from ‘www.icc-cricket.com’

### i) Top 10 ODI teams in men’s cricket along with the records for matches, points and rating

In [191]:
def find_Top10_ODI_MensTeam(url):
    url = url+'/rankings/mens/team-rankings/odi'
    return Top10_teams(url)

In [192]:
find_Top10_ODI_MensTeam('https://www.icc-cricket.com')

### ii) Top 10 ODI Batsmen in men along with the records of their team and rating.

In [193]:
def find_Top10_ODI_MenBatting(url):
    url = url+'/rankings/mens/player-rankings/odi/batting'
    return find_Top10_player(url)

In [194]:
find_Top10_ODI_MenBatting('https://www.icc-cricket.com')

### iii) Top 10 ODI bowlers in men along with the records of their team and rating

In [195]:
def find_Top10_ODI_MenBowling(url):
    url = url+'/rankings/mens/player-rankings/odi/bowling'
    return find_Top10_player(url)

In [196]:
find_Top10_ODI_MenBowling('https://www.icc-cricket.com')

## 6.Python program to scrape cricket rankings from ‘www.icc-cricket.com’

### i) Top 10 ODI teams in women’s cricket along with the records for matches, points and rating

In [197]:
def find_Top10_ODI_WomensTeam(url):
    url = url+'/rankings/womens/team-rankings/odi'
    return Top10_teams(url)

In [198]:
find_Top10_ODI_WomensTeam('https://www.icc-cricket.com')

### ii) Top 10 women’s ODI players along with the records of their team and rating. 

In [199]:
def find_Top10_WomensODI_player(url):
    url = url+'/rankings/womens/player-rankings/odi/batting'
    return find_Top10_player(url)

In [200]:
find_Top10_WomensODI_player('https://www.icc-cricket.com')

### iii) Top 10 women’s ODI all-rounder along with the records of their team and rating.

In [201]:
def find_Top10_WomensODI_AllRounder(url):
    url = url+'/rankings/womens/player-rankings/odi/all-rounder'
    return find_Top10_player(url)

In [202]:
find_Top10_WomensODI_AllRounder('https://www.icc-cricket.com')

## 7. Python program to scrape details of all the mobile phones under Rs. 20,000 listed on Amazon.in. The scraped data should include Product Name, Price, Image URL and Average Rating.

In [169]:
def Amazon_mbl_20k(url):
    from bs4 import BeautifulSoup
    import requests
    import pandas as pd  
    import time
    Product_Name, Price, Average_Rating, Image_Url= [],[],[],[] 
    url = url+'/s?i=electronics&bbn=1389401031&rh=n%3A976419031%2Cn%3A1389401031%2Cn%3A1389432031%2Cp_36%3A-2000000&dc&qid=1610187018&rnid=1389401031&ref=sr_nr_n_3'

    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')


    name = soup.find_all('span', class_='a-size-base-plus a-color-base a-text-normal')
    rating = soup.find_all('a', class_='a-popover-trigger a-declarative')
    price = soup.find_all('span', {'class':['a-price-whole','a-offscreen']})
    imageurl = soup.find_all('img', class_='s-image')


    for i in name:
        Product_Name.append(i.text.replace('\n',''))
    for j in rating:
        Average_Rating.append(j.span.text.replace('\n',''))
    for k in price:
        Price.append(''.join(filter(lambda j: j.isdigit(), k.text.replace('\n','').replace(' ', ''))))
    for l in imageurl:
        Image_Url.append(l.attrs['src'])



    amazon_lst = pd.DataFrame({})
    amazon_lst['Mobile Name'] = pd.Series(Product_Name) #To handle NaN values(If there is any), we converted into Series
    amazon_lst['Mobile Price'] = pd.Series(Price)
    amazon_lst['Average Rating'] = pd.Series(Average_Rating)
    amazon_lst['Image URL'] = pd.Series(Image_Url)
    #amazon_lst = amazon_lst.sort_values(by=['Mobile Price'], ascending =False)

    compression_opts = dict(method='zip', archive_name='Amazon_Mbls_under_20k.csv')
    return amazon_lst.to_csv('Amazon_Mbls_under_20k.zip', index=False, compression=compression_opts)

In [170]:
Amazon_mbl_20k('https://www.amazon.in')

## 8. A python program to extract information about the local weather from the National Weather Service website of USA, https://www.weather.gov/ for the city, San Francisco

need to extract data about 7 day
extended forecast display for the city. The data should include period, short description, temperature and
description. 

In [74]:
def weather_Forecast(url):
    from bs4 import BeautifulSoup
    import requests
    import pandas as pd  

    Period, Short_Description, Temperature, Description= [],[],[],[]
    url = url+'/MapClick.php?lat=37.777120000000025&lon=-122.41963999999996#.X_pvTugzbIU'

    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    for i in soup.find_all('div', class_='col-sm-2 forecast-label'):
        Period.append(i.text)

    for i in soup.find_all('div', class_='col-sm-10 forecast-text'):
        desc = i.text.replace('\n','')
        temp = (''.join(filter(lambda j: j.isdigit(), desc)))
        Temperature.append(temp[:2])
        Description.append(desc)
        desc_split = desc.split(',')
        if len(desc_split)>2:
            Short_Description.append((' '.join(desc_split[0:2])))
        else:
            Short_Description.append(desc_split[0])

    weather_df = pd.DataFrame({})
    weather_df['Period'] = pd.Series(Period) #To handle NaN values(If there is any), we converted into Series
    weather_df['Short Description'] = pd.Series(Short_Description)
    weather_df['Temperature'] = pd.Series(Temperature)
    weather_df['Description'] = pd.Series(Description)


    compression_opts = dict(method='zip', archive_name='weather.csv')
    return weather_df.to_csv('weather_report.zip', index=False, compression=compression_opts)

In [75]:
weather_Forecast('https://forecast.weather.gov')

## 9. python program to scrape ‘software developer’ job listings from ‘Monster.com’. It should include all the jobs listed for the next 5 pages.

In [82]:
from bs4 import BeautifulSoup
import requests
import pandas as pd 

url = 'https://www.monsterindia.com/srp/results?query=Software%20Developer&searchId=16f7d6f8-55fe-4a1a-9d4f-11d0934edaa7'
page = requests.get(url)
page.content

b'<HTML><HEAD>\n<TITLE>Access Denied</TITLE>\n</HEAD><BODY>\n<H1>Access Denied</H1>\n \nYou don\'t have permission to access "http&#58;&#47;&#47;www&#46;monsterindia&#46;com&#47;srp&#47;results&#63;" on this server.<P>\nReference&#32;&#35;18&#46;8dc72c31&#46;1610272896&#46;d29a3a9d\n</BODY>\n</HTML>\n'

In [83]:
page.status_code

403

#### Since the Monster.com is not allowing Web Scrapping, We are doing same exercise on Naukri.com

#### Naukri.com was not possible with BeautifulSoup, So we used Selenium for this

In [92]:
#pip install selenium #Please Don't Run Already Installed

Note: you may need to restart the kernel to use updated packages.Collecting selenium



## 10. python program to scrape ‘data scientist’ job listings for location ‘New Delhi’ from ‘Monster.com’. It should include all the jobs listed for the next 5 pages.

#### Since the Monster.com is not allowing Web Scrapping, I am doing same exercise on Naukri.com

#### Naukri.com was not possible with BeautifulSoup, So we used Selenium for this

In [147]:
#Common Function to get required data for question 9 and 10 in webscrapping assignment
def Job_report(url):
    import selenium
    import pandas as pd
    from selenium import webdriver
    import time

    driver = webdriver.Chrome(r'C:\Users\udayakiran\Desktop\chromedriver.exe') # system local Chromedriver location path
    driver.get(url)
    #time.sleep(15) #15sec sleep mode Time to open browser and load required url site
    
    Job_title, Company_Name= [],[]
    if("data-scientist" in url) and ('new%20delhi' in url):
        City=[]
        

    for i in range(0,5):#range For First five pages loop
        for i in driver.find_elements_by_xpath("//a[@class='title fw500 ellipsis']"):
            Job_title.append(i.text)
        for j in driver.find_elements_by_xpath("//a[@class='subTitle ellipsis fleft']"):
            Company_Name.append(j.text)
            if("data-scientist" in url) and ('new%20delhi' in url):
                for k in driver.find_elements_by_xpath("//li[@class='fleft grey-text br2 placeHolderLi location']/span[1]"):
                    City.append(k.text)
        driver.find_element_by_xpath("//a[@class='fright fs14 btn-secondary br2']").click()
        time.sleep(3)

    job_lst = pd.DataFrame({})
    job_lst['Job Title'] = pd.Series(Job_title)
    job_lst['Company Name'] = pd.Series(Company_Name)
    if("data-scientist" in url) and ('new%20delhi' in url):
        job_lst['Location'] = pd.Series(City)
    
    if("data-scientist" in url) and ('new%20delhi' in url):
        compression_opts = dict(method='zip', archive_name='Job_report_of_DS_Delhi_Location.csv')
        return job_lst.to_csv('Job_report_of_DS_Delhi_Location.zip', index=False, compression=compression_opts)
    else:
        compression_opts = dict(method='zip', archive_name='Job_Report_of_5pages.csv')
        return job_lst.to_csv('Job_Report_of_5pages.zip', index=False, compression=compression_opts)        
    

In [148]:
def Job_report_of_5pages(url):
    url = url+"/software-developer-jobs?k=software%20developer"
    return Job_report(url)

In [149]:
def Job_report_of_DS_Delhi_Location(url):
    url = url+"/data-scientist-jobs-in-new-delhi?k=data%20scientist&l=new%20delhi"
    return Job_report(url)

In [150]:
Job_report_of_5pages('https://www.naukri.com')

In [ ]:
Job_report_of_DS_Delhi_Location('https://www.naukri.com')